## AtLAST sensitivity calculator

This is a short tutorial on using the sensitivity calculator and customising the input parameters.

First and foremost, let's install the `atlast_sc` package. You can find more details in the GitHub Installation Guide: https://github.com/ukatc/AtLAST_sensitivity_calculator/blob/main/docs/source/user_guide/python_package_installation.rst

In [43]:
try:
    import atlast_sc
except:
    !pip install git+https://github.com/ukatc/AtLAST_sensitivity_calculator.git

Once you have `atlast_sc` installed and correctly running, you can import the main calculator tool (along with some useful packages) as:

In [44]:
from atlast_sc.calculator import Calculator

from astropy import coordinates as coord
from astropy import units as u

calc = Calculator()

The last step initializes the calculator. This is pre-configured with default values for all user input parameters. Below is an overview of their default values.

In [54]:
print(calc.user_input)

t_int: 4984.110697 s
sensitivity: 0.42 mJy
bandwidth: 100.0 MHz
obs_freq: 100.0 GHz
n_pol: 2.0 GHz
weather: 25.0 GHz
elevation: 45.0 deg


Finally, obtaining a sensitivity estimate for a given integration time is as easy as follows:

In [46]:
t_int = 150*u.s
calculated_sensitivity = calc.calculate_sensitivity(t_int)
print(calculated_sensitivity)
print(calculated_sensitivity.to(u.uJy))

2.4210151133810287 mJy
2421.0151133810286 uJy


The calculator will take care of all the necessary unit conversions, so you can define your input `t_int` in any units you prefer.

If, alternatively, you would like to estimate an integration time for a given target sensitivity, you can simply do:

In [52]:
sensitivity = 0.42*u.mJy
calculated_t_int = calc.calculate_t_integration(sensitivity)
print(calculated_t_int)
print(calculated_t_int.to(u.h))

4984.110696615099 s
1.3844751935041941 h


### Customizing the input parameters

To change the calculator parameters, there are two options. You can either change the individual values once the calculator is initialized,

In [53]:
calc_pars = Calculator()
calc_pars.bandwidth = 2.00*u.GHz
calc_pars.obs_freq = 90*u.GHz
calc_pars.weather = 50

or provide an input dictionary,

In [51]:
user_input = {'bandwidth': {'value': 200.00, 'unit':  'MHz'},
               'obs_freq': {'value':  90.00, 'unit':  'GHz'},
                'weather': {'value':  50.00, 'unit': 'none'}}
calc_dict = Calculator(user_input=user_input)

or by uploading an yaml-formatted file (see [here](https://github.com/ukatc/AtLAST_sensitivity_calculator/blob/main/docs/source/user_guide/providing_input_data.rst#reading-data-from-an-input-file) for details):

In [ ]:
from atlast_sc.utils import FileHelper
user_input = FileHelper.read_from_file('<directory>', '<file name>')
calc_input = Calculator(user_input=user_input)

### Some useful functions

I am adding here some utilities that I defined while running the validation tests and that could be generally useful.

One of the input parameters is the elevation of the observation. If you want to reproduce what you would get, for instance, with the ALMA sensitivity calculator, you should input the maximum elevation for a given declination. To estimate the former, you can use the `maxelev` function below:

In [61]:
from astropy import coordinates as coords
from astropy import units as u

import numpy as np

def maxelev(dec,site='ALMA'):
  dec = dec.to(u.deg).value
  lat = coords.EarthLocation.of_site(site)

  rdec = np.deg2rad(dec)
  rlat = np.deg2rad(lat.to_geodetic().lat.value)

  factor = np.arcsin(np.sin(rdec)*np.sin(rlat)+np.cos(rdec)*np.cos(rlat))
  return np.rad2deg(factor)*u.deg

declination = -10.00*u.deg


calc = Calculator()
calc.elevation = maxelev(dec=declination)

The `weather` parameter refers to the relative humidity and is define in percentile values. If you want to consider octiles, instead, you can obtain the corresponding percentile using the following conversion

In [63]:
def oct2perc(octile):
    refoct = {i: 12.5*i for i in range(1,8)}
    return refoct[int(octile)]

calc.weather = oct2perc(3)